In [1]:
%pip install torch zkstats==0.1.7


[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
from zkstats.core import (
    setup,
)
from zkstats.computation import computation_to_model

cwd = os.getcwd()


# Paths to the output files
output_dir = f"{cwd}/out"
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Shared by the data provider beforehand
data_shape = {'x': 7, 'y': 7}
data_commitment_path = f"{output_dir}/data_commitment.json"

User select the columns

In [4]:
# FIXME: this should be provided by users
# selected_columns = ["x", "y"]
selected_columns = ["x"]

## User-defined Computation

A computation should be of type `TComputation`. For example, the following code snippet defines a computation that computes the sum of the private data.

```python
def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.cat([out_0.unsqueeze(0), out_1.unsqueeze(0)]).reshape(-1,1))
```

FIXME: The following code snippet is entirely from the user. You MUST check
1. the code only performs zkstats-related operations.
2. the computation must not leak any information about the private data.

In [5]:
# This is just a dummy computation. Replace it with user's computation
import torch
from zkstats.computation import State

def computation(state: State, x: list[torch.Tensor]):
    # out_0 = state.median(x[0])
    # out_1 = state.median(x[1])
    # # return state.mean(torch.cat([out_0.unsqueeze(0), out_1.unsqueeze(0)]).reshape(-1,1)), out_0
    # return out_0, out_1
    return state.mean(x[0]), state.median(x[0])

Get proof, settings, and precal_witness from the data provider

In [6]:
proof_path = f"{output_dir}/model.pf"
settings_path = f"{output_dir}/settings.json"
precal_witness_path = f"{output_dir}/precal_witness.json"

## Verify the proof to ensure it is correct
NOTE: The following section is to illustrate what should be done on the user (data consumer) side. This step is not required by the data provider.

In [7]:
import json
import numpy as np
from typing import Dict


def create_dummy(shape_info: Dict[str, int], dummy_data_path: str) -> None:
    """
    Create a dummy data file with randomized data based on the provided shape information.

    Parameters:
    - shape_info (dict): A dictionary where keys are column names and values are the number of elements (shape).
    - dummy_data_path (str): The path to save the dummy data file.
    """
    dummy_data = {}
    for col, length in shape_info.items():
        # Generate random data for each column
        dummy_data[col] = np.round(np.random.uniform(0, 100, length), 1).tolist()

    with open(dummy_data_path, 'w') as f:
        json.dump(dummy_data, f)


from zkstats.core import verifier_define_calculation, verifier_verify

verifier_model_path = f"{output_dir}/verifier_model.onnx"
verifier_compiled_model_path = f"{output_dir}/verifier_model.compiled"
verifier_vk_path = f"{output_dir}/verifier_model.vk"
verifier_pk_path = f"{output_dir}/verifier_model.pk"
dummy_data_path = f"{output_dir}/dummy_data.json"
sel_dummy_data_path = f"{output_dir}/sel_dummy_data.json"

# NOTE: generate the verifier model with the `precal_witness_path` provided by the prover
_, verifier_model = computation_to_model(computation, precal_witness_path, isProver=False)
# Determine which srs to use with the logrows in the settings.json
with open(settings_path, "r") as f:
    settings = json.load(f)
logrows = int(settings["run_args"]["logrows"])
srs_path = f'~/.ezkl/srs/kzg{logrows}.srs'

# create dummy data with the same shape as the original data
create_dummy(data_shape, dummy_data_path)
# generate the verifier model given the dummy data and the selected columns
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)
# generate the verification key
setup(verifier_model_path, verifier_compiled_model_path, settings_path, verifier_vk_path, verifier_pk_path)
# verify the proof
verifier_verify(proof_path, settings_path, verifier_vk_path, selected_columns, data_commitment_path)

/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:Where, node name: /Where_1): Y has inconsistent type tensor(float) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/

==== setting up ezkl ====
Time setup: 0.5691819190979004 seconds


[51.5, 46.25]

Print the file paths. You should share the following files back to the user for them to verify the proof. You **SHOULD NOT** share more files otherwise data might be leaked.

In [8]:
print("Model onnx:\t\t", verifier_model_path)
print("Settings:\t\t", settings_path)
print("Proof:\t\t\t", proof_path)
print("Verification key:\t", verifier_vk_path)
print("Srs path:\t\t", srs_path)

Model onnx:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/verifier_model.onnx
Settings:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/settings.json
Proof:			 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.pf
Verification key:	 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/verifier_model.vk
Srs path:		 ~/.ezkl/srs/kzg11.srs
